In [1]:
import pandas as pd
import requests
from datetime import datetime
from jsonpath import jsonpath


def get_zt_stock_bank(date: str = None) -> pd.DataFrame:
    """
    获取指定日期涨停股票行情

    Parameters
    ----------
    date : str
        指定日期，默认为当前日期 格式形如 ``'20220506'``

    Returns
    -------
    DataFrame
        指定日期涨停股票行情

    Fields
    ------
    ``['日期', '股票代码', '股票名称', '涨跌幅', '成交额', '封板资金', '流通市值', '最新价', '换手率','首次封板时间', '最后封板时间', '炸板次数', '连扳数', '所属行业']``
    """
    if date is None:
        date = datetime.today().strftime('%Y%m%d')
    params = (
        ('ut', 'b2884a393a59ad64002292a3e90d46a5'),
        ('pn', '1'),
        ('pz', '500'),
        ('po', '1'),
        ('np', '1'),
        ('fields', 'f12,f13,f14,f62'),
        ('fid','f62'),
        ('fs','m:90+t:2'),
    )

    response = requests.get(
        'https://push2.eastmoney.com/api/qt/clist/get', params=params,  verify=False)
    fields = {
        'f12': '板块编码',
        'f13': '上市板块',
        'f14': '板块名称',
        'f62': '主力资金净流入'
    }
    items = jsonpath(response.json(), '$..diff[:]')
    if not items:
        df = pd.DataFrame(
            columns=['日期']+list(fields.values())+['统计天数', '涨停次数'])
        return df
    df = pd.DataFrame(items)
    
    
    df: pd.DataFrame = df.rename(columns=fields)[fields.values()]
    
    df['主力资金净流入'] /= 100000000
    
    return df

df1 = get_zt_stock_bank()
df1

ModuleNotFoundError: No module named 'pandas'